# Grain Process

In [1]:
import yt.units as u
import numpy as np
atoms = ['H', 'C', 'O']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994}

## Grain model

In [2]:
r = 1000 * u.angstrom
V = 4 / 3 * np.pi * r**3
rho = 3 * u.gram / u.cm**3
Ns = 1e6
G_M_ratio = 100
nH = 2e4 / u.cm**3
T = 10 * u.Kelvin
nd = nH * u.mass_hydrogen / G_M_ratio / rho / V

In [3]:
nd.in_cgs()

2.663832342956152e-08 cm**(-3)

In [4]:
def grain_spe(spe):
    if spe == ' ' or len(spe) == 0:
        return ('')
    t = ''
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
    return(grain_spe)

## Accretion

$$R_{\mathrm{acc}}(i)=\overline{\sigma_{d}n_{d}}\langle v(i)\rangle n(i) = \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)$$

$$k_i= \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)=6.0557\sqrt{\frac{T\ (\text{K})}{\mu}}\times10^{-14}\text{ s}^{-1}$$

In [5]:
def mu(spe):
    t = ''
    m = 0
    for i in spe:
        if i.isdigit():
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            m += mass[t]
    return(m)

def acc(spe, label=0):
    react = '{},{},{},NONE,NONE,6.0557e-14*sqrt(Tgas/{:f})'.format(
        label, spe, grain_spe(spe), mu(spe))
    return(react)

In [6]:
LIST_complex = [
    'H', 'O', 'C', 'H2', 'CH', 'OH', 'CO', 'H2O', 'CH2', 'CH3', 'HOC', 'HCO',
    'CO2', 'CHOH', 'H2CO', 'CH4', 'CH2OH', 'CH3OH'
]
for i, spe in enumerate(LIST_complex, 4432):
    print(acc(spe, label=i))

4432,H,H_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/1.007900)
4433,O,O_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.999400)
4434,C,C_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/12.010700)
4435,H2,H_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/2.015800)
4436,CH,C_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/13.018600)
4437,OH,O_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/17.007300)
4438,CO,C_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/28.010100)
4439,H2O,H_dustH_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/18.015200)
4440,CH2,C_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/14.026500)
4441,CH3,C_dustH_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.034400)
4442,HOC,H_dustO_dustC_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/29.018000)
4443,HCO,H_dustC_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/29.018000)
4444,CO2,C_dustO_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/44.009500)
4445,CHOH,C_dustH_dustO_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/30.025900)
4446,H2CO,H_dustH_dustC_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/30.025900)
44

## Evaporation

### Thermal Evaporation

$$k_{\mathrm{eva, thermal}}(i)=\nu_{i} \exp \left[-E_{D} / kT\right]$$

$$\begin{aligned} \nu_{i} &=\left(2 \rho_{s} E_{D} / \pi^{2} m_{i}\right)^{1 / 2} \\ &=2.4 \times 10^{12} \mathrm{Hz}\left(\frac{\rho_{s}}{10^{15} \mathrm{cm}^{-2}}\right)^{1 / 2}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \end{aligned}$$

In [7]:
rhos = Ns/4/np.pi/r**2
np.sqrt(rhos.in_cgs()/1e15*u.cm**2)

0.8920620580763855 dimensionless

As a result,

$$k_{\mathrm{eva, thermal}}(i)=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \exp \left[-E_{D} / T\right]$$

### Cosmic Ray Desorption

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=\zeta(70 \mathrm{K}) \cdot \nu_{i} \exp \left[-E_{D}/(70 \mathrm{K})\right]$$

$\zeta(70\text{K})\sim3.16 \times 10^{-19}$ (Hasegawa & Herbst, 1993), thus

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=6.76\times10^{-7}\text{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp\left[-E_{D} / (70\text{K})\right]$$

### Photodesorption

- R. Visser et al. 2011

Laboratory experiments on the photodesorption of H2O, CO and CO2 all produce a yield of $Y\approx10^{-3}$ molecules per grain per incident UV photon (Öberg et al. 2007, 2009a), while the yield for N2 is an order of magnitude lower (Öberg et al. 2009b). For all other ice species in our network, whose photodesorption yields have not yet been deter- mined experimentally or theoretically, the yield is set to $10^{−3}$. The photodesorption rate is

$$
R_{\mathrm{phdes}}(X)=\pi a^{2} n_d f(X) Y(X) F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

where

$$
f(X)=\frac{n_{\mathrm{s}}(X)}{\max \left(n_{\mathrm{ice}}, N_{s} n_{d}\right)}
$$

Here $n_{d}=2.66\times10^{-8}$ cm$^{-3}$, $N_s=10^6$, thus only when $n_{\mathrm{ice}}>2.66\times10^{-2}$ cm$^{-3}$ do we take $f(X)$ as ${n_{\mathrm{s}}(X)}/{n_{\mathrm{ice}}}$ and in other cases

$$
k_{\mathrm{eva,UV}}(i)\approx10^{-9}\pi a^{2}  F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

Photodesorption occurs even in strongly shielded regions because of cosmic-ray-induced photons, which is accounted for by setting a lower limit of $10^4$ cm$^{−2}$ s$^{−1}$ to the product $F_{0} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)$.

In [8]:
(1e-3*np.pi*r**2).in_cgs()

3.1415926535897937e-13 cm**2

Thus,

$$
k_{\mathrm{eva,UV}}(i)=3.14\times 10^{-19} \text{Hz}\cdot \frac{F_{0}}{\text{cm}^{-2}\text{ s}^{-1}} \exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

The UV background we use is taken from Draine 1978:

$$F(E)=\left[1.658 \times 10^{6}(E / \mathrm{eV})-2.152 \times 10^{5}(E / \mathrm{eV})^{2}+6.919 \times 10^{3}(E / \mathrm{eV})^{3}\right] \text { photons } \mathrm{cm}^{-2} \mathrm{s}^{-1}\ \mathrm{sr}^{-1}\ \mathrm{eV}^{-1}$$

Between 5 eV and 13.6 eV,

$$F=2.42 \times 10^{8} \text { photons } \mathrm{cm}^{-2} \mathrm{s}^{-1}$$

Thus,

$$
k_{\mathrm{eva,UV}}(i)=7.59\times 10^{-11} \text{Hz}\exp \left(-\tau_{\mathrm{UV}, \mathrm{eff}}\right)
$$

In [9]:
(1.658e6/2*(13.6**2-5**2)-2.152e5/3*(13.6**3-5**3)+6.919e3/4*(13.6**4-5**4))*4*np.pi

241594323.46166772

In [10]:
def eva(spe, Eads, label=0):
    kth = 2.14e12 / u.s * np.sqrt(Eads / 350 / u.Kelvin) / np.sqrt(
        mu(spe) / mu('H'))
    kcos = 6.76e-7 / u.s * np.sqrt(Eads / 350 / u.Kelvin) / np.sqrt(
        mu(spe) / mu('H')) * np.exp(-Eads / 70 / u.Kelvin)
    kuv = 7.59e-11 / u.s
    
    react = '{},{},{},NONE,NONE,{:e}*exp(-{:f}/Tgas)+{:e}+{}*exp(-user_Av)'.format(
        label, grain_spe(spe), spe, kth, Eads, kcos, kuv)
    return (react)

In [11]:
calorie = 41858000 * u.erg
kcal = 1e3 * calorie
Eads = {}
Eads['H'] = 350 * u.Kelvin
Eads['H2'] = 450 * u.Kelvin
Eads['C'] = 800 * u.Kelvin
Eads['O'] = 800 * u.Kelvin
Eads['CH'] = 1.3 * kcal / 6.02e23 / u.boltzmann_constant
Eads['OH'] = 2.5 * kcal / 6.02e23 / u.boltzmann_constant
Eads['CO'] = 2.4 * kcal / 6.02e23 / u.boltzmann_constant
Eads['CH2'] = 1.9 * kcal / 6.02e23 / u.boltzmann_constant
Eads['H2O'] = 2.9 * kcal / 6.02e23 / u.boltzmann_constant
Eads['HOC'] = 3.5 * kcal / 6.02e23 / u.boltzmann_constant
Eads['HCO'] = 1510 * u.Kelvin
Eads['CO2'] = 2500 * u.Kelvin
Eads['CH3'] = 2.3 * kcal / 6.02e23 / u.boltzmann_constant
Eads['CHOH'] = 3.8 * kcal / 6.02e23 / u.boltzmann_constant
Eads['H2CO'] = 1760 * u.Kelvin
Eads['CH4'] = 1360 * u.Kelvin
Eads['CH2OH'] = 4.3 * kcal / 6.02e23 / u.boltzmann_constant
Eads['CH3OH'] = 4.1 * kcal / 6.02e23 / u.boltzmann_constant
for item, i in enumerate(Eads.keys()):
    print(i)
    print(Eads[i])
    kth = 2.14e12 / u.s * np.sqrt(Eads[i] / 350 / u.Kelvin) / np.sqrt(
        mu(i) / mu('H')) * np.exp(-Eads[i] / 10 / u.Kelvin)
    kcos = 6.76e-7 / u.s * np.sqrt(Eads[i] / 350 / u.Kelvin) / np.sqrt(
        mu(i) / mu('H')) * np.exp(-Eads[i] / 70 / u.Kelvin)
    kuv = 7.59e-11
    print('{:e} {:e} {:e}'.format(kth, kcos, kuv))

H
350.0 K
1.349295e-03 4.554852e-09 7.590000e-11
H2
450.0 K
4.911559e-08 8.752071e-10 7.590000e-11
C
800.0 K
1.691573e-23 3.221193e-12 7.590000e-11
O
800.0 K
1.465626e-23 2.790931e-12 7.590000e-11
CH
654.6996593219958 K
3.003188e-17 2.230839e-11 7.590000e-11
OH
1259.0378063884534 K
2.067631e-43 4.819390e-15 7.590000e-11
CO
1208.6762941329152 K
2.429079e-41 7.555119e-15 7.590000e-11
CH2
956.8687328552245 K
2.634868e-30 3.467060e-13 7.590000e-11
H2O
1460.4838554106059 K
3.859307e-52 2.837306e-16 7.590000e-11
HOC
1762.6529289438347 K
2.516483e-65 3.277235e-18 7.590000e-11
HCO
1510.0 K
2.186636e-54 1.120531e-16 7.590000e-11
CO2
2500.0 K
2.310281e-97 8.439194e-23 7.590000e-11
CH3
1158.314781877377 K
4.994441e-39 2.072846e-14 7.590000e-11
CHOH
1913.737465710449 K
7.074887e-72 3.877854e-19 7.590000e-11
H2CO
1760.0 K
3.223057e-65 3.343687e-18 7.590000e-11
CH4
1360.0 K
9.123786e-48 1.219091e-15 7.590000e-11
CH2OH
2165.54502698814 K
8.580825e-83 1.111727e-20 7.590000e-11
CH3OH
2064.8220024770635

In [12]:
for item, i in enumerate(Eads.keys(), 4450):
    print(eva(i, Eads[i], label=item))

4450,H_dust,H,NONE,NONE,2.140000e+12*exp(-350.000000/Tgas)+4.554852e-09+7.59e-11*exp(-user_Av)
4451,H_dustH_dust,H2,NONE,NONE,1.715817e+12*exp(-450.000000/Tgas)+8.752071e-10+7.59e-11*exp(-user_Av)
4452,C_dust,C,NONE,NONE,9.372367e+11*exp(-800.000000/Tgas)+3.221193e-12+7.59e-11*exp(-user_Av)
4453,O_dust,O,NONE,NONE,8.120478e+11*exp(-800.000000/Tgas)+2.790931e-12+7.59e-11*exp(-user_Av)
4454,C_dustH_dust,CH,NONE,NONE,8.143805e+11*exp(-654.699659/Tgas)+2.230839e-11+7.59e-11*exp(-user_Av)
4455,O_dustH_dust,OH,NONE,NONE,9.880756e+11*exp(-1259.037806/Tgas)+4.819390e-15+7.59e-11*exp(-user_Av)
4456,C_dustO_dust,CO,NONE,NONE,7.543730e+11*exp(-1208.676294/Tgas)+7.555119e-15+7.59e-11*exp(-user_Av)
4457,C_dustH_dustH_dust,CH2,NONE,NONE,9.485058e+11*exp(-956.868733/Tgas)+3.467060e-13+7.59e-11*exp(-user_Av)
4458,H_dustH_dustO_dust,H2O,NONE,NONE,1.033992e+12*exp(-1460.483855/Tgas)+2.837306e-16+7.59e-11*exp(-user_Av)
4459,H_dustO_dustC_dust,HOC,NONE,NONE,8.950311e+11*exp(-1762.652929/Tgas)+3.277235e-18

## Surface reactions

$$k_{ij}=\kappa_{ij}(R_{\mathrm{diff},i}+R_{\mathrm{diff},j})/n_d$$

$$\kappa_{ij}=\exp[-2(1 \overset{\circ}{\text{A}}/\hbar)(2\mu E_a)^{1/2}]$$

$$R_{\mathrm{diff},i}=N_s^{-1}\nu_o\exp[-E_b/T]\approx N_s^{-1}\nu_i\exp[-0.3E_D/T]=2.14 \times 10^{6} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp[-0.3E_D/T]$$

except for $\mathrm{H}$ and $\mathrm{H}_2$, of which

$$R_{\mathrm{diff},i}=\nu_{i} \exp \left[-(2 \overset{\circ}{\text{A}} / \hbar)\left(2 m E_{b}\right)^{1 / 2}\right]=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp \left[-(2 \overset{\circ}{\text{A}} / \hbar)\left(2 m E_{b}\right)^{1 / 2}\right]$$

In [13]:
import pandas as pd

In [14]:
def Rdiff(spe, T=10 * u.K):
    if spe in ['H2', 'H']:
        return (2.14e6 / u.s * np.sqrt(Eads[spe] / 350 / u.K) /
                np.sqrt(mu(spe) / mu('H')) * np.exp(
                    (-(2 * 0.3 * Eads[spe] * mu(spe) / mu('H') *
                       u.mass_hydrogen * u.boltzmann_constant)**(1 / 2) *
                     (2 * u.angstrom / u.hbar)).in_cgs()))
    else:
        #2.14e6 - Ns=1e6
        #3.6e6 - ns=1.4e15
        return (2.14e6 / u.s * np.sqrt(Eads[spe] / 350 / u.K) /
                np.sqrt(mu(spe) / mu('H')) * np.exp(-0.3 * Eads[spe] / T))

In [15]:
Rdiff('H'), Rdiff('H2'), Rdiff('C')

(32810.63869021714 1/s, 2113.2928673734577 1/s, 3.538194816255683e-05 1/s)

In [16]:
def kappa(spei, spej, Ea):
    muij = mu(spei) * mu(spej) / (mu(spei) + mu(spej)) * (u.mass_hydrogen /
                                                          mu('H'))
    return np.exp((-2 * (u.angstrom / u.hbar) * (2 * muij * Ea * u.boltzmann_constant)**(1 / 2)).in_cgs())

In [17]:
nd.in_cgs()

2.663832342956152e-08 cm**(-3)

In [21]:
pd.read_csv('../../Notes/grain_reaction.csv').fillna('')

,R1,R2,P1,P2,k_ij,Ea
0,H,H,H2,,3.880000e+12,0
1,H,C,CH,,1.940000e+12,0
2,H,O,OH,,1.940000e+12,0
3,H,CH,CH2,,1.940000e+12,0
4,H,OH,H2O,,1.940000e+12,0
5,H,CO,HOC,,2.570000e+06,1000
6,H,CO,HCO,,2.570000e+06,1000
7,H,CH2,CH3,,1.940000e+12,0
8,H,CH3,CH4,,1.940000e+12,0
9,H,HOC,CHOH,,1.940000e+12,0


In [22]:
reaction = pd.read_csv('../../Notes/grain_reaction.csv').fillna('')
for i in reaction.index:
    rec = reaction.loc[i]
    print(rec)
    k = kappa(rec['R1'], rec['R2'],
              rec['Ea'] * u.K) * (Rdiff(rec['R1']) + Rdiff(rec['R2'])) / nd
    print('{:e}'.format(k.in_cgs()))

R1             H
R2             H
P1            H2
P2              
k_ij    3.88e+12
Ea             0
Name: 0, dtype: object
2.463416e+12
R1             H
R2             C
P1            CH
P2              
k_ij    1.94e+12
Ea             0
Name: 1, dtype: object
1.231708e+12
R1             H
R2             O
P1            OH
P2              
k_ij    1.94e+12
Ea             0
Name: 2, dtype: object
1.231708e+12
R1             H
R2            CH
P1           CH2
P2              
k_ij    1.94e+12
Ea             0
Name: 3, dtype: object
1.231708e+12
R1             H
R2            OH
P1           H2O
P2              
k_ij    1.94e+12
Ea             0
Name: 4, dtype: object
1.231708e+12
R1             H
R2            CO
P1           HOC
P2              
k_ij    2.57e+06
Ea          1000
Name: 5, dtype: object
3.883765e+06
R1             H
R2            CO
P1           HCO
P2              
k_ij    2.57e+06
Ea          1000
Name: 6, dtype: object
3.883765e+06
R1             H
R2           CH2


In [23]:
for item, i in enumerate(reaction.index, 4468):
    rec = reaction.loc[i]
    ki = (kappa(rec['R1'], rec['R2'], rec['Ea'] * u.K) * Rdiff(rec['R1']) / nd).in_cgs()
    kj = (kappa(rec['R1'], rec['R2'], rec['Ea'] * u.K) * Rdiff(rec['R2']) / nd).in_cgs()
    if not rec['R1'] in ['H', 'H2']:
        ki /= np.exp(-0.3 * Eads[rec['R1']] / 10 / u.K)
        ki_string = '{:e}*exp(-0.3*{}/Tgas)'.format(ki, Eads[rec['R1']])
    else:
        ki_string = '{:e}'.format(ki)
    if not rec['R2'] in ['H', 'H2']:
        kj /= np.exp(-0.3 * Eads[rec['R2']] / 10 / u.K)
        kj_string = '{:e}*exp(-0.3*{}/Tgas)'.format(kj, Eads[rec['R2']])
    else:
        kj_string = '{:e}'.format(kj)
    print('{},{},{},{},{},NONE,NONE,{}+{}'.format(item, grain_spe(rec['R1']),
                                                  grain_spe(rec['R2']),
                                                  grain_spe(rec['P1']),
                                                  grain_spe(rec['P2']), ki_string,
                                                  kj_string))

4468,H_dust,H_dust,H_dustH_dust,,NONE,NONE,1.231708e+12+1.231708e+12
4469,H_dust,C_dust,C_dustH_dust,,NONE,NONE,1.231708e+12+3.518377e+13*exp(-0.3*800.0/Tgas)
4470,H_dust,O_dust,O_dustH_dust,,NONE,NONE,1.231708e+12+3.048420e+13*exp(-0.3*800.0/Tgas)
4471,H_dust,C_dustH_dust,C_dustH_dustH_dust,,NONE,NONE,1.231708e+12+3.057176e+13*exp(-0.3*654.6996593219958/Tgas)
4472,H_dust,O_dustH_dust,H_dustH_dustO_dust,,NONE,NONE,1.231708e+12+3.709226e+13*exp(-0.3*1259.0378063884534/Tgas)
4473,H_dust,C_dustO_dust,H_dustO_dustC_dust,,NONE,NONE,3.883765e+06+8.929445e+07*exp(-0.3*1208.6762941329152/Tgas)
4474,H_dust,C_dustO_dust,H_dustC_dustO_dust,,NONE,NONE,3.883765e+06+8.929445e+07*exp(-0.3*1208.6762941329152/Tgas)
4475,H_dust,C_dustH_dustH_dust,C_dustH_dustH_dustH_dust,,NONE,NONE,1.231708e+12+3.560681e+13*exp(-0.3*956.8687328552245/Tgas)
4476,H_dust,C_dustH_dustH_dustH_dust,C_dustH_dustH_dustH_dustH_dust,,NONE,NONE,1.231708e+12+3.784006e+13*exp(-0.3*1158.314781877377/Tgas)
4477,H_dust,H_dustO_dustC_du